# Classificador de notícias usando o algoritmo Multinomial Naive Bayes

* Neste projeto foi construído um classificador de notícias para determinar sua categoria utilizando o algoritmo Multinomial Naive Bayes.


* Sua documentação pode ser encontrada no seguinte link: 
http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html


* O classificador Multinomial Naive Bayes é recomendado quando se trabalha com variáveis discretas. 


* A base de dados utilizada pode ser encontrada a seguir: 
http://qwone.com/~jason/20Newsgroups/

In [1]:
# Importação dos pacotes utilizados
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# Definindo as categorias das notícias para limitar os tipos que quero utilizar
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

In [3]:
# Criando a base de dados de treino 
twenty_train = fetch_20newsgroups(subset='train', categories = categories, shuffle=True, random_state = 42)

In [5]:
# Classes da base que foram definidas
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [7]:
# Tamanho da base de dados
len(twenty_train.data)

2257

In [12]:
# Visualizando alguns dados
print('\n'.join(twenty_train.data[0].split('\n')[:3]))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton


In [13]:
# Visualizando as respostas dos dados acima
print(twenty_train.target_names[twenty_train.target[0]])

comp.graphics


In [15]:
# O Scikit-Learn registrou as respostas como qualitativa a fim de aumentar a velocidade
twenty_train.target[:10]

array([1, 1, 3, 3, 3, 3, 3, 2, 2, 2])

###Construindo o Bag of Words (Saco de Palavras)

In [21]:
# Tokenizando as palavras, ou seja, reduzindo as palavras
count_vector = CountVectorizer()
X_train_counts = count_vector.fit_transform(twenty_train.data)
count_vector.vocabulary_.get(u'algorithm')
X_train_counts.shape

(2257, 35788)

In [25]:
# De ocorrências para frequências resultando em uma matriz esparsa - Tf-idf
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

In [26]:
# Criando o modelo Multinomial Naive Bayes
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [28]:
# Realizando as previsões com base em frases criadas
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vector.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


O modelo previu as frases da maneira correta

#### É possível fazer este processo através do uso de Pipeline

* Nas linhas a seguir será utilizado Pipeline com o mesmo objetivo

In [38]:
# Criação do Pipeline com as transformações necessárias nos dados e o modelo Multinomial
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

In [39]:
# Fit do modelo
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

In [47]:
# Verificando a Acurácia do modelo
twenty_test = fetch_20newsgroups(subset = 'test', categories = categories, shuffle = True, random_state = 42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
print('Acurácia obtida: {}'.format(np.mean(predicted == twenty_test.target)))

Acurácia obtida: 0.8348868175765646


In [48]:
# Métricas obtidas
print(metrics.classification_report(twenty_test.target, predicted, target_names = twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.97      0.60      0.74       319
         comp.graphics       0.96      0.89      0.92       389
               sci.med       0.97      0.81      0.88       396
soc.religion.christian       0.65      0.99      0.78       398

              accuracy                           0.83      1502
             macro avg       0.89      0.82      0.83      1502
          weighted avg       0.88      0.83      0.84      1502



In [46]:
# Confusion Matrix
metrics.confusion_matrix(twenty_test.target, predicted)

array([[192,   2,   6, 119],
       [  2, 347,   4,  36],
       [  2,  11, 322,  61],
       [  2,   2,   1, 393]])